In [2]:
import sys
sys.path.append("../src")

import logging
from pathlib import Path
import numpy as np
import torch
from natsort import natsorted
from torch import nn
from matplotlib import pyplot as plt
from dataset.data_loading import BasicDataset
from dataset.dataset_interface import DatasetInterface
from networks.UNet.unet_model import UNet
import open3d as o3d
from utils.visualization_utils import visualize_depth, to_rgb
from utils.transformation_utils import imgs_to_pcd, rs_ci, unnormalize_depth
import yaml
from utils.transformation_utils import imgs_to_pcd, pcd_to_imgs, pcd_to_imgs_old, rs_ci, zv_ci

root_path = Path("../")

class Args:
    image_path = root_path / "resources/images/raw/ycb_video/10022022"

args = Args()
debug = False

transformation = np.array(
    [[0.99807603, -0.01957923, -0.05882929,  0.17911331],
        [0.0212527, 0.99938319, 0.02795643, -0.01932425],
        [0.05824564, -0.02915292, 0.99787652, -0.03608739],
        [0., 0., 0., 1.]]
)

files = DatasetInterface.get_files_by_path(args.image_path)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
idx = np.random.randint(len(files), size=1).item()
rs_rgb, rs_depth, zv_rgb, zv_depth = DatasetInterface.load(files[idx])[:4]

if debug:
    rs_pcd = imgs_to_pcd(rs_rgb, rs_depth, rs_ci)
# 2D to 3D
zv_pcd = imgs_to_pcd(zv_rgb, zv_depth, zv_ci)

if debug:
    o3d.visualization.draw_geometries([zv_pcd, rs_pcd], "raw pcds")

# transform
zv_pcd.transform(transformation)

if debug:
    o3d.visualization.draw_geometries([zv_pcd, rs_pcd], "calibrated pcds")

# 3D to 2D
zv_rgb, zv_depth, _, _ = pcd_to_imgs(zv_pcd, rs_ci)
zv_rgb_old, zv_depth_old, _, _ = pcd_to_imgs_old(zv_pcd, rs_ci)

zv_rgb[:] = (0,0,0)
zv_rgb_old[:] = (0,255,0)

test_pcd = imgs_to_pcd(zv_rgb, zv_depth, rs_ci)
test_pcd_old = imgs_to_pcd(zv_rgb_old, zv_depth_old, rs_ci)
o3d.visualization.draw_geometries([test_pcd_old,test_pcd], "test pcds")

# total pixels: 1752845, # unique pixels: 447787


### Visualise image pairs

In [4]:
files = DatasetInterface.get_files_by_path( root_path /'resources/images/calibrated_new/not-cropped/ycb_video/10022022')
idx = np.random.randint(len(files), size=1).item()
rs_rgb, rs_depth, zv_rgb, zv_depth = DatasetInterface.load(files[idx])[:4]
zv_pcd = imgs_to_pcd(zv_rgb, zv_depth, rs_ci)
rs_pcd = imgs_to_pcd(rs_rgb, rs_depth, rs_ci)
o3d.visualization.draw_geometries([zv_pcd, rs_pcd], "test pcds")